# LangGraph Pendulum Agent Walkthrough

Replay the pendulum sandbox logic and fetch the reasoning transcript from the site.

## 1. Imports

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import requests
from math import pi, sqrt

pio.renderers.default = 'plotly_mimetype'

## 2. Simulation Helpers

In [ ]:
def simulate_trials(lengths, noise_sigma=0.02, gravity=9.81, seed=17):
    rng = np.random.default_rng(seed)
    rows = []
    for length in lengths:
        true_period = 2 * pi * sqrt(length / gravity)
        noisy = true_period * (1 + rng.uniform(-1, 1) * noise_sigma)
        rows.append({'length': length, 'period': noisy})
    return pd.DataFrame(rows)

def fit_parameters(df):
    x = np.log(df['length'])
    y = np.log(df['period'])
    A = np.vstack([x, np.ones(len(x))]).T
    p, intercept = np.linalg.lstsq(A, y, rcond=None)[0]
    return np.exp(intercept), p


In [ ]:
lengths = [0.5, 0.8, 1.2, 1.6, 2.1]
noise_sigma = 0.02
trials = simulate_trials(lengths, noise_sigma)
k, p = fit_parameters(trials)
trials.head(), (k, p)

## 3. Notebook View

In [ ]:
def reasoning_steps(trials_df, k, p, noise_sigma):
    latest = trials_df.iloc[-1]
    coverage = trials_df['length'].max() - trials_df['length'].min()
    descriptor = 'broad' if coverage > 1.2 else 'moderate' if coverage > 0.6 else 'narrow'
    residual = abs(k * latest['length'] ** p - latest['period'])
    next_length = round(trials_df['length'].max() + 0.35, 2)
    return [
        {'role': 'planner', 'text': f"Queue contains {len(trials_df)} trials with {descriptor} coverage (σ={noise_sigma:.2f})."},
        {'role': 'observation', 'text': f"Latest @ {latest['length']:.2f} m produced T={latest['period']:.3f}s; residual {residual:.3f}s."},
        {'role': 'analysis', 'text': f"Fit → k={k:.3f}, p={p:.3f}; exponent drift within ±0.01."},
        {'role': 'action', 'text': f"Schedule next trial near {next_length:.2f} m to widen coverage."}
    ]
reasoning_steps(trials, k, p, noise_sigma)

## 4. Fetch Site Transcript

## 5. Plot Loss Surface

In [ ]:
ks = np.linspace(5.9, 6.7, 30)
ps = np.linspace(0.45, 0.55, 30)
zz = np.zeros((len(ps), len(ks)))
for i, k_ in enumerate(ks):
    for j, p_ in enumerate(ps):
        preds = k_ * trials['length'].values ** p_
        zz[j, i] = np.sqrt(np.mean((preds - trials['period'].values) ** 2))
go.Figure(data=go.Heatmap(x=ks, y=ps, z=zz, colorscale='Magma', colorbar=dict(title='RMSE'))).show()